In [7]:
import sys
sys.path.append("../../Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages")

In [8]:
from Levenshtein import distance as lev

In [9]:
class Dataset:
    def __init__(self, data:dict):
        self._data=data
        self._original=data
    def reset_data(self):
        self.data=self.original
    def keys(self):
        return self._data.keys()
    def values(self):
        return self._data.values()
    def items(self):
        return self._data.items()

In [10]:
strategy_labels={0:"random",1:"single_word_syn",2:"n_word_syn",3:"single_word_ant",4:"n_word_ant",5:"misspell",6:"reorder",7:"addition",8:"deletion"}
strategy_ids={"random":0,"single_word_syn":1,"n_word_syn":2,"single_word_ant":3,"n_word_ant":4,"misspell":5,"reorder":6,"addition":7,"deletion":8}

In [15]:
class MutagenDataset(Dataset):
    def __init__(self, data:dict, operator:Operator, mutations=[], strategy="random"):
        super().__init__(data)
        self._mutations=mutations
        self._strategy=strategy
        self._stratid=strategy_ids[strategy]
        self._operator=operator
    def reset_data(self):
        super().reset_data()
        self._mutations=[]
    def appendMutations(self,mutation:list):
        self._mutations.append(mutation)
    def computeLevenshtein(self):
        distances=[]
        for i in range(0,len(self._data)):
            distances.append(lev(self._original[i],self._data[i]))
        average_dist=sum(distances)/len(self._data)
        return distances,average_dist

In [14]:
class Operator:
    def __init__(self,stratid):
        self.strategy=strategy_ids[stratid]
    def mutate(self,data:dict):
        if self.strategy==0:
            pass
        elif self.strategy==1:
            pass
        elif self.strategy==2:
            pass
        elif self.strategy==3:
            pass
        elif self.strategy==4:
            pass
        elif self.strategy==5:
            pass
        elif self.strategy==6:
            pass
        elif self.strategy==7:
            pass
        elif self.strategy==8:
            pass
